# List images from a BDSA folder

This notebook uses the Brain Digital Slide Archive (BDSA) REST API to fetch every `largeImage` item inside the Girder folder `67ddb0782fc8ce397c5ef7fb` and display basic metadata.
Set your credentials in `etl/.env` (or export `BDSA_BASE`, `BDSA_API_KEY`, etc.) before running.


In [1]:
from pathlib import Path
import os
from typing import Dict, Iterator, List

import requests
from dotenv import load_dotenv

ENV_PATH = Path("etl/.env")
if ENV_PATH.exists():
    load_dotenv(ENV_PATH)

BDSA_BASE = os.getenv("BDSA_BASE", "").rstrip("/")
BDSA_API_KEY = os.getenv("BDSA_API_KEY", "")
DEFAULT_FOLDER_ID = "67ddb0782fc8ce397c5ef7fb"
FOLDER_ID = (os.getenv("BDSA_FOLDER_ID", DEFAULT_FOLDER_ID) or DEFAULT_FOLDER_ID).strip()

if not BDSA_BASE:
    raise RuntimeError("Set the BDSA_BASE environment variable before running this notebook.")

HEADERS = {"Girder-Token": BDSA_API_KEY} if BDSA_API_KEY else {}

print(f"BDSA_BASE: {BDSA_BASE}")
print(f"Folder ID: {FOLDER_ID}")
print("Authenticated request" if HEADERS else "Anonymous request (no API token found)")


ModuleNotFoundError: No module named 'dotenv'

In [ ]:
from typing import Dict, Iterator


def thumbnail_url(item_id: str, width: int = 512, height: int = 512) -> str:
    root = BDSA_BASE.rsplit("/api/v1", 1)[0]
    return f"{root}/api/v1/item/{item_id}/tiles/thumbnail?width={width}&height={height}"


def iter_folder_images(folder_id: str, *, limit: int = 200) -> Iterator[Dict[str, str]]:
    offset = 0
    while True:
        params = {
            "folderId": folder_id,
            "limit": limit,
            "offset": offset,
            "sort": "updated",
            "sortdir": 1,
        }
        resp = requests.get(f"{BDSA_BASE}/item", params=params, headers=HEADERS, timeout=60)
        resp.raise_for_status()
        items = resp.json()
        if not items:
            break

        for raw in items:
            if not raw.get("largeImage"):
                continue
            yield {
                "name": raw.get("name"),
                "item_id": raw.get("_id"),
                "updated": raw.get("updated"),
                "size": raw.get("size"),
                "thumbnail": thumbnail_url(raw["_id"]),
            }
        offset += limit


In [ ]:
images = list(iter_folder_images(FOLDER_ID))
print(f"Found {len(images)} largeImage item(s) in folder {FOLDER_ID}.")

try:
    import pandas as pd
except ImportError:
    pd = None

if pd and images:
    display(pd.DataFrame(images))
else:
    from pprint import pprint
    pprint(images[:10])
